In [333]:
from dateutil.relativedelta import relativedelta
from argon2 import PasswordHasher
from datetime import datetime
from faker import Faker
import pandas as pd
import random as rd
import re

SEED = 29
Faker.seed(SEED)
rd.seed(SEED)

ph = PasswordHasher()
fk = Faker('pt-BR')
QNT = 10

In [334]:
def generate_addresses():
    complements = ['Casa 2', 'Apto. 162', 'Casa B', 'Galpão A', None, None, None, None, None, None]
    raw_addresses = []

    for ad in [fk.address() for _ in range(2 * QNT - 1)]:
        split_ad = ad.split('\n')
        street_address = split_ad[0].split(',')
        street = street_address[0]
        number = int(street_address[1]) if len(street_address) > 1 else 0
        temp, state = split_ad[2].split('/')
        postal_code, city = temp.split(' ', maxsplit=1)
        postal_code = postal_code.replace('-', '')
        complement = rd.choice(complements)

        raw_addresses.append(('Brasil', state, city, street, number, postal_code, complement))

    return raw_addresses

In [335]:
addresses = pd.DataFrame(
    data=generate_addresses(),
    columns=['country', 'state', 'city', 'street', 'number', 'postal_code', 'complement']
)

display(addresses)

,country,state,city,street,number,postal_code,complement
0,Brasil,MT,Monteiro,Trevo de da Paz,9,66001735,None
1,Brasil,SE,Jesus,Estrada de Machado,73,98386448,Apto. 162
2,Brasil,RS,Leão,Lagoa Sá,89,52425186,None
3,Brasil,AL,Silveira,Vale Leonardo Duarte,14,66826240,None
4,Brasil,SC,Alves,Vereda Moreira,0,08022753,None
5,Brasil,AM,Pinto,Alameda José Correia,3,63338353,None
6,Brasil,RO,Monteiro do Sul,Viela Leão,6,92461606,Apto. 162
7,Brasil,PE,Cavalcanti de Ramos,Distrito Cavalcante,89,53707393,None
8,Brasil,RN,Sampaio,Área Porto,90,40143531,None
9,Brasil,MA,Gonçalves dos Dourados,Colônia da Cunha,15,33634485,None


In [351]:
gen_date = lambda: fk.date_between(datetime.now() - relativedelta(years=50), datetime.now() - relativedelta(years=12))
gen_password = lambda size: fk.password(length=size)

users = pd.DataFrame({
    'cpf': [fk.cpf() for _ in range(QNT)],
    'profile_pic': [None] * QNT,
    'email': [fk.email() for _ in range(QNT)],
    'username': [fk.name() for _ in range(QNT)],
    'phone': [fk.phone_number() for _ in range(QNT)],
    'birthday': [gen_date() for _ in range(QNT)],
    'raw_password': [gen_password(8) for _ in range(QNT)],
    'address_id': [(i % addresses.size + 1) for i in range(QNT)],
})

users['password'] = users['raw_password'].apply(lambda x: ph.hash(x))
users['phone'] = users['phone'].apply(lambda x: re.sub(r'[^0-9]', '', x))
users.to_csv('csv/test_logins.csv', columns=['email', 'raw_password'])
users.drop(columns=['raw_password'], inplace=True)

display(users)

,cpf,profile_pic,email,username,phone,birthday,address_id,password
0,965.831.702-21,None,pietra21@example.com,Arthur Gabriel das Neves,4115628290,1992-05-03,1,"$argon2id$v=19$m=65536,t=3,p=4$CYtzj+UkzQMk/Ie..."
1,481.297.530-14,None,da-cunhapedro-lucas@example.com,João Miguel Moreira,558164586470,1997-07-03,2,"$argon2id$v=19$m=65536,t=3,p=4$IVuaNOFuz4ef8kZ..."
2,095.836.724-83,None,gomesevelyn@example.org,Sra. Ana Júlia Cassiano,1166370437,1979-02-28,3,"$argon2id$v=19$m=65536,t=3,p=4$SWo7fUMEnznCvcp..."
3,897.136.250-21,None,cassianojuliana@example.org,Mariane Lima,558420920855,1987-08-11,4,"$argon2id$v=19$m=65536,t=3,p=4$6cdsjfRUaoYbnv1..."
4,671.859.423-73,None,ymelo@example.org,Enzo Gabriel Pires,03009779487,1977-05-18,5,"$argon2id$v=19$m=65536,t=3,p=4$pLRwlwU/caqVsgO..."
5,039.542.671-52,None,vpimenta@example.net,Maria Eduarda da Rocha,7191356786,1990-06-04,6,"$argon2id$v=19$m=65536,t=3,p=4$Jh794yzJOPOZD6Y..."
6,271.398.605-21,None,tcarvalho@example.net,Benjamin da Rosa,5506114682655,1984-11-27,7,"$argon2id$v=19$m=65536,t=3,p=4$0/X+3Pw0tgKfpx2..."
7,948.712.605-85,None,vitor-hugo71@example.net,Ayla Peixoto,556110492355,1986-01-18,8,"$argon2id$v=19$m=65536,t=3,p=4$FP10CGLG5Ts3pt+..."
8,986.720.543-00,None,barrosfernanda@example.com,Agatha Almeida,5166158009,1999-03-21,9,"$argon2id$v=19$m=65536,t=3,p=4$VS1QEkWxHXZRzR3..."
9,209.754.863-65,None,pedro62@example.org,Amanda Melo,5507106396497,1988-10-28,10,"$argon2id$v=19$m=65536,t=3,p=4$v444p3IWWweww4V..."
